In [1]:
import os
import datetime
import sys
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Point

# Clean Arrest Data

In [9]:
df_ytd = pd.read_csv('/content/NYPD_Arrest_Data__Year_to_Date__20250418.csv')
df_his = pd.read_csv('/content/NYPD_Arrests_Data__Historic__20250418.csv')
df = pd.concat([df_his, df_ytd], ignore_index=True)

df.columns = df.columns.str.lower()

print("total datasets：", df.shape)


total datasets： (6057267, 20)


In [10]:
df.head()

,arrest_key,arrest_date,pd_cd,pd_desc,ky_cd,ofns_desc,law_code,law_cat_cd,arrest_boro,arrest_precinct,jurisdiction_code,age_group,perp_sex,perp_race,x_coord_cd,y_coord_cd,latitude,longitude,lon_lat,new georeferenced column
0,279197226,12/19/2023,105.0,STRANGULATION 1ST,106.0,FELONY ASSAULT,PL 1211200,F,M,18,0.0,25-44,M,WHITE,988210.0,218129.0,40.765390,-73.985702,POINT (-73.985702 40.76539),NaN
1,278761840,12/09/2023,105.0,STRANGULATION 1ST,106.0,FELONY ASSAULT,PL 1211300,F,K,67,0.0,25-44,M,BLACK,997897.0,175676.0,40.648859,-73.950820,POINT (-73.95082 40.648859),NaN
2,278506761,12/05/2023,153.0,RAPE 3,104.0,RAPE,PL 1302503,F,K,77,0.0,25-44,M,BLACK,1003509.0,185018.0,40.674496,-73.930571,POINT (-73.9305713255961 40.6744956865259),NaN
3,278436408,12/03/2023,157.0,RAPE 1,104.0,RAPE,PL 1303501,F,B,46,0.0,45-64,M,BLACK,1011755.0,250279.0,40.853598,-73.900577,POINT (-73.9005768807295 40.8535983673823),NaN
4,278248753,11/29/2023,660.0,(null),NaN,(null),PL 2407800,M,Q,104,0.0,<18,M,WHITE HISPANIC,1011456.0,194092.0,40.699373,-73.901881,POINT (-73.901881 40.699373),NaN


In [11]:
def clean_arrest_data(df):
    boro_dict = {'B': 'Bronx', 'S': 'Staten Island', 'K': 'Brooklyn', 'M': 'Manhattan', 'Q': 'Queens'}
    perp_sex_dict = {'F': 'Female', 'M': 'Male'}
    law_cat_cd_dict = {'F': 'Felony', 'M': 'Misdemeanor', 'V': 'Violation', 'I': 'Traffic Infraction'}

    df['arrest_boro'] = df.arrest_boro.replace(boro_dict)
    df['perp_sex'] = df.perp_sex.replace(perp_sex_dict)
    df['law_cat_cd'] = df['law_cat_cd'].replace(law_cat_cd_dict)
    df['perp_race'] = df['perp_race'].apply(lambda x: x.title())
    df['ofns_desc'] = df['ofns_desc'].astype(str).apply(lambda x: x.title())

    # change latitude and longitude from string to numeric
    df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
    df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
    df = df.dropna(subset=['latitude', 'longitude'])

    return df

df_cleaned = clean_arrest_data(df)
print("Cleaned dataset", df_cleaned.shape)


Cleaned dataset (6057262, 20)


In [12]:
df_cleaned.to_csv('/content/nyc_arrests_cleaned_basic.csv', index=False)
print("✅ Saved to /content/nyc_arrests_cleaned_basic.csv")

✅ Saved to /content/nyc_arrests_cleaned_basic.csv


In [13]:
df_cleaned.head()

,arrest_key,arrest_date,pd_cd,pd_desc,ky_cd,ofns_desc,law_code,law_cat_cd,arrest_boro,arrest_precinct,jurisdiction_code,age_group,perp_sex,perp_race,x_coord_cd,y_coord_cd,latitude,longitude,lon_lat,new georeferenced column
0,279197226,12/19/2023,105.0,STRANGULATION 1ST,106.0,Felony Assault,PL 1211200,Felony,Manhattan,18,0.0,25-44,Male,White,988210.0,218129.0,40.765390,-73.985702,POINT (-73.985702 40.76539),NaN
1,278761840,12/09/2023,105.0,STRANGULATION 1ST,106.0,Felony Assault,PL 1211300,Felony,Brooklyn,67,0.0,25-44,Male,Black,997897.0,175676.0,40.648859,-73.950820,POINT (-73.95082 40.648859),NaN
2,278506761,12/05/2023,153.0,RAPE 3,104.0,Rape,PL 1302503,Felony,Brooklyn,77,0.0,25-44,Male,Black,1003509.0,185018.0,40.674496,-73.930571,POINT (-73.9305713255961 40.6744956865259),NaN
3,278436408,12/03/2023,157.0,RAPE 1,104.0,Rape,PL 1303501,Felony,Bronx,46,0.0,45-64,Male,Black,1011755.0,250279.0,40.853598,-73.900577,POINT (-73.9005768807295 40.8535983673823),NaN
4,278248753,11/29/2023,660.0,(null),NaN,(Null),PL 2407800,Misdemeanor,Queens,104,0.0,<18,Male,White Hispanic,1011456.0,194092.0,40.699373,-73.901881,POINT (-73.901881 40.699373),NaN


# Clean Neighborhood Data

In [14]:
import pandas as pd

# load neighborhood
nta_df = pd.read_csv('/content/2020_Neighborhood_Tabulation_Areas__NTAs__20250418.csv')

print("orign_shape:", nta_df.shape)

nta_df_cleaned = nta_df[['NTA2020', 'NTAName', 'BoroName']].copy()

nta_df_cleaned = nta_df_cleaned.dropna()

print("Cleaned_shape:", nta_df_cleaned.shape)

# Save
nta_df_cleaned.to_csv('/content/nta_cleaned.csv', index=False)
print("✅ Neighborhood (NTA) saved to /content/nta_cleaned.csv")



orign_shape: (262, 12)
Cleaned_shape: (262, 3)
✅ Neighborhood (NTA) saved to /content/nta_cleaned.csv


In [15]:
nta_df_cleaned.head()

,NTA2020,NTAName,BoroName
0,BK0101,Greenpoint,Brooklyn
1,BK0102,Williamsburg,Brooklyn
2,BK0103,South Williamsburg,Brooklyn
3,BK0104,East Williamsburg,Brooklyn
4,BK0201,Brooklyn Heights,Brooklyn


# Spatial Join

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [3]:
# load cleaned arrest datasets
df_cleaned = pd.read_csv('/content/nyc_arrests_cleaned_basic.csv')

# load NTA shapefile
nta_gdf = gpd.read_file('/content/geo_export_000877df-36cd-4d0a-a214-e2ffcb3422a8.shp')

print("Shapefile字段：", nta_gdf.columns)



<ipython-input-3-836cac436324>:2: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cleaned = pd.read_csv('/content/nyc_arrests_cleaned_basic.csv')


Shapefile字段： Index(['borocode', 'boroname', 'countyfips', 'nta2020', 'ntaname', 'ntaabbrev',
       'ntatype', 'cdta2020', 'cdtaname', 'shape_leng', 'shape_area',
       'geometry'],
      dtype='object')


In [4]:
nta_gdf = nta_gdf.to_crs(epsg=4326)
nta_gdf = nta_gdf[nta_gdf.geometry.type.isin(['Polygon', 'MultiPolygon'])]
nta_gdf = nta_gdf[~nta_gdf.is_empty & nta_gdf.is_valid]


In [5]:
# 3. arrest datasets to GeoDataFrame
gdf_arrest = gpd.GeoDataFrame(
    df_cleaned,
    geometry=gpd.points_from_xy(df_cleaned.longitude, df_cleaned.latitude),
    crs="EPSG:4326"
)


# Spatial Join
gdf_joined = gpd.sjoin(
    gdf_arrest,
    nta_gdf[['ntaname', 'geometry']],
    how="left",
    predicate="within"
)

gdf_joined = gdf_joined.rename(columns={"ntaname": "neighborhood"})
gdf_joined['neighborhood'] = gdf_joined['neighborhood'].fillna("Unknown")


In [6]:
# Save
gdf_joined.drop(columns=['geometry']).to_csv('/content/nyc_arrests_with_neighborhood.csv', index=False)

print("✅ Save to /content/nyc_arrests_with_neighborhood.csv")


✅ Save to /content/nyc_arrests_with_neighborhood.csv


# Check

In [7]:
df_final = pd.read_csv('/content/nyc_arrests_with_neighborhood.csv')

print("Shape:", df_final.shape)
df_final.head()


<ipython-input-7-88410811bd1b>:1: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df_final = pd.read_csv('/content/nyc_arrests_with_neighborhood.csv')


Shape: (6057262, 22)


,arrest_key,arrest_date,pd_cd,pd_desc,ky_cd,ofns_desc,law_code,law_cat_cd,arrest_boro,arrest_precinct,...,perp_sex,perp_race,x_coord_cd,y_coord_cd,latitude,longitude,lon_lat,new georeferenced column,index_right,neighborhood
0,279197226,12/19/2023,105.0,STRANGULATION 1ST,106.0,Felony Assault,PL 1211200,Felony,Manhattan,18,...,Male,White,988210.0,218129.0,40.765390,-73.985702,POINT (-73.985702 40.76539),NaN,128.0,Hell's Kitchen
1,278761840,12/09/2023,105.0,STRANGULATION 1ST,106.0,Felony Assault,PL 1211300,Felony,Brooklyn,67,...,Male,Black,997897.0,175676.0,40.648859,-73.950820,POINT (-73.95082 40.648859),NaN,54.0,East Flatbush-Erasmus
2,278506761,12/05/2023,153.0,RAPE 3,104.0,Rape,PL 1302503,Felony,Brooklyn,77,...,Male,Black,1003509.0,185018.0,40.674496,-73.930571,POINT (-73.9305713255961 40.6744956865259),NaN,27.0,Crown Heights (North)
3,278436408,12/03/2023,157.0,RAPE 1,104.0,Rape,PL 1303501,Felony,Bronx,46,...,Male,Black,1011755.0,250279.0,40.853598,-73.900577,POINT (-73.9005768807295 40.8535983673823),NaN,84.0,Mount Hope
4,278248753,11/29/2023,660.0,(null),NaN,(Null),PL 2407800,Misdemeanor,Queens,104,...,Male,White Hispanic,1011456.0,194092.0,40.699373,-73.901881,POINT (-73.901881 40.699373),NaN,176.0,Ridgewood


In [8]:
print(df_final.columns)

Index(['arrest_key', 'arrest_date', 'pd_cd', 'pd_desc', 'ky_cd', 'ofns_desc',
       'law_code', 'law_cat_cd', 'arrest_boro', 'arrest_precinct',
       'jurisdiction_code', 'age_group', 'perp_sex', 'perp_race', 'x_coord_cd',
       'y_coord_cd', 'latitude', 'longitude', 'lon_lat',
       'new georeferenced column', 'index_right', 'neighborhood'],
      dtype='object')


In [9]:
print(df_final['neighborhood'].value_counts())

neighborhood
East Harlem (North)                      139348
Midtown-Times Square                     120039
Mott Haven-Port Morris                    99575
Jamaica                                   97861
Chelsea-Hudson Yards                      96807
                                          ...  
Spring Creek Park                             7
Calvert Vaux Park                             6
Mount Hebron & Cedar Grove Cemeteries         4
Shirley Chisholm State Park                   3
Montefiore Cemetery                           3
Name: count, Length: 258, dtype: int64


In [10]:
df_final[['arrest_boro', 'perp_sex', 'law_cat_cd', 'neighborhood']].sample(10)

,arrest_boro,perp_sex,law_cat_cd,neighborhood
3190625,Bronx,Male,Felony,Yankee Stadium-Macombs Dam Park
5890553,Manhattan,Male,Misdemeanor,Manhattanville-West Harlem
2167281,Brooklyn,Female,Felony,Fort Greene
3106479,Manhattan,Female,Misdemeanor,Harlem (North)
790757,Bronx,Female,Felony,Morris Park
2187885,Queens,Male,Misdemeanor,Queensbridge-Ravenswood-Dutch Kills
4016633,Bronx,Male,Misdemeanor,Allerton
5092737,Bronx,Male,Felony,Longwood
1037474,Bronx,Male,Misdemeanor,University Heights (North)-Fordham
2167046,Queens,Male,Misdemeanor,Jackson Heights


# With Precinct Number

In [9]:
import pandas as pd

arrest_df = pd.read_csv('/content/nyc_arrests_with_neighborhood.csv')
precincts_df = pd.read_csv('/content/police_precincts.csv')


precincts_cleaned = precincts_df[['Precinct Number', 'Precinct Name', 'Borough']].copy()
precincts_cleaned = precincts_cleaned.rename(columns={'Precinct Number': 'arrest_precinct'})


precincts_cleaned['arrest_precinct'] = precincts_cleaned['arrest_precinct'].astype(int)
arrest_df['arrest_precinct'] = arrest_df['arrest_precinct'].astype(int)


<ipython-input-9-c86c0c6c8e05>:3: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  arrest_df = pd.read_csv('/content/nyc_arrests_with_neighborhood.csv')


In [10]:
arrest_full_df = arrest_df.merge(
    precincts_cleaned,
    how='left',
    on='arrest_precinct'
)

print("✅ Combined shape:", arrest_full_df.shape)

✅ Combined shape: (6057262, 24)


In [11]:
arrest_full_df.to_csv('/content/nyc_arrests_full_cleaned.csv', index=False)
print("✅ saved to /content/nyc_arrests_full_cleaned.csv")

✅ saved to /content/nyc_arrests_full_cleaned.csv


# Final Check

In [12]:
# 读最新的完整数据
df_final = pd.read_csv('/content/nyc_arrests_full_cleaned.csv')

# 1. 查看行数和列数
print(f"✅ Shape (rows, columns): {df_final.shape}")

# 2. 列出所有字段名
print("\n✅ Columns:")
print(df_final.columns.tolist())

# 3. 随机看几行数据（确认格式）
print("\n✅ Random 5 samples:")
display(df_final.sample(5))

# 4. 检查 neighborhood 字段的唯一值数量
print("\n✅ Unique neighborhoods:", df_final['neighborhood'].nunique())

# 5. 检查 precinct 相关字段
if 'Precinct Name' in df_final.columns and 'Borough' in df_final.columns:
    print("\n✅ Precinct Name and Borough columns are PRESENT.")
else:
    print("\n❌ Precinct Name and Borough columns are MISSING!")

# 6. 统计一下每个 Borough 的逮捕数量
print("\n✅ Arrest count by Borough:")
print(df_final['Borough'].value_counts())


<ipython-input-12-48ffc1fc76e4>:2: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df_final = pd.read_csv('/content/nyc_arrests_full_cleaned.csv')


✅ Shape (rows, columns): (6057262, 24)

✅ Columns:
['arrest_key', 'arrest_date', 'pd_cd', 'pd_desc', 'ky_cd', 'ofns_desc', 'law_code', 'law_cat_cd', 'arrest_boro', 'arrest_precinct', 'jurisdiction_code', 'age_group', 'perp_sex', 'perp_race', 'x_coord_cd', 'y_coord_cd', 'latitude', 'longitude', 'lon_lat', 'new georeferenced column', 'index_right', 'neighborhood', 'Precinct Name', 'Borough']

✅ Random 5 samples:


,arrest_key,arrest_date,pd_cd,pd_desc,ky_cd,ofns_desc,law_code,law_cat_cd,arrest_boro,arrest_precinct,...,x_coord_cd,y_coord_cd,latitude,longitude,lon_lat,new georeferenced column,index_right,neighborhood,Precinct Name,Borough
1896764,153882504,06/10/2016,569.0,"MARIJUANA, SALE 4 & 5",235.0,Dangerous Drugs,PL 2213500,Misdemeanor,Manhattan,25,...,1000318.0,232713.0,40.805413,-73.941961,POINT (-73.94196119299994 40.805412948000026),NaN,148.0,East Harlem (North),25th Precinct,Manhattan
2114472,146080410,09/12/2015,808.0,TAX LAW,364.0,Other State Laws (Non Penal La,TAX181400G,Felony,Brooklyn,79,...,998237.0,187028.0,40.680023,-73.949573,POINT (-73.94957309899996 40.68002258100006),NaN,9.0,Bedford-Stuyvesant (West),79th Precinct,Brooklyn
3353623,85244358,06/13/2012,101.0,ASSAULT 3,344.0,Assault 3 & Related Offenses,PL 1204501,Misdemeanor,Manhattan,26,...,995807.0,236438.0,40.815644,-73.958249,POINT (-73.95824925199997 40.81564409200007),NaN,143.0,Manhattanville-West Harlem,26th Precinct,Manhattan
901891,213901759,06/06/2020,101.0,ASSAULT 3,344.0,Assault 3 & Related Offenses,PL 1200001,Misdemeanor,Bronx,46,...,1009065.0,250064.0,40.853016,-73.910301,POINT (-73.91030143299997 40.85301622600008),NaN,83.0,University Heights (South)-Morris Heights,46th Precinct,Bronx
4927997,50845173,09/06/2008,511.0,"CONTROLLED SUBSTANCE, POSSESSION 7",235.0,Dangerous Drugs,PL 2200300,Misdemeanor,Bronx,44,...,1006817.0,240067.0,40.825583,-73.918461,POINT (-73.91846102199997 40.82558340700007),NaN,78.0,Concourse-Concourse Village,44th Precinct,Bronx



✅ Unique neighborhoods: 258

✅ Precinct Name and Borough columns are PRESENT.

✅ Arrest count by Borough:
Borough
Brooklyn         1678760
Manhattan        1608414
Bronx            1384623
Queens           1164200
Staten Island     221262
Name: count, dtype: int64
